In [2]:
import os
from PIL import Image
from PIL import Image, ImageDraw, ImageFont
import random
import numpy as np
import cv2

In [3]:
image_path = '../../images/raw_random'
image_files = os.listdir(image_path)
img = Image.open(os.path.join(image_path, image_files[0]))
img = img.resize((640, 640))
img.show()

In [5]:
def get_Rx(theta):
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def get_Ry(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def get_Rz(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

def rotate_2d_point_in_3d(point, theta_x, theta_y, theta_z):
    point = np.array([point[0], point[1], 0])
    point = np.dot(get_Rx(theta_x), point)
    point = np.dot(get_Ry(theta_y), point)
    point = np.dot(get_Rz(theta_z), point)
    return point[:2]

def translate_points_to_image(points, image_size):
    point1, point2, point3, point4 = points

    min_x = min(point1[0], point2[0], point3[0], point4[0])
    min_y = min(point1[1], point2[1], point3[1], point4[1])
    max_x = max(point1[0], point2[0], point3[0], point4[0])
    max_y = max(point1[1], point2[1], point3[1], point4[1])

    dx = max(0, -min_x, max_x - image_size[0])
    dy = max(0, -min_y, max_y - image_size[1])

    translated_points = [(point[0] + 1.1 * dx, point[1] + 1.1 * dy) for point in points]

    return translated_points

In [6]:
image_path = '../../images/raw_random'
image_files = os.listdir(image_path)
img = Image.open(os.path.join(image_path, image_files[1]))
img = img.resize((640, 640))

# randomly pick a box of dimension 52x11 in the image
size = 6
dimensions = (52 * size, 11 * size)
x = 400
y = 200

point1 = (x, y)
point2 = (x + dimensions[0], y)
point3 = (x + dimensions[0], y + dimensions[1])
point4 = (x, y + dimensions[1])

theta_x = np.deg2rad(random.uniform(-60, 60))
theta_y = np.deg2rad(random.uniform(-60, 60))
theta_z = np.deg2rad(random.uniform(-10, 10))

point1 = rotate_2d_point_in_3d(point1, theta_x, theta_y, theta_z)
point2 = rotate_2d_point_in_3d(point2, theta_x, theta_y, theta_z)
point3 = rotate_2d_point_in_3d(point3, theta_x, theta_y, theta_z)
point4 = rotate_2d_point_in_3d(point4, theta_x, theta_y, theta_z)


print(point1, point2, point3, point4)

point1, point2, point3, point4 = translate_points_to_image([point1, point2, point3, point4], img.size)

print(point1, point2, point3, point4)
draw = ImageDraw.Draw(img)
draw.polygon([point1, point2, point3, point4], outline='red', width=3)



lp_img = 'license_plate.png'
lp_img = Image.open(lp_img)
# lp_img = lp_img.resize((640, 640))

box_points_lp = [(0, 0), (lp_img.width, 0), (lp_img.width, lp_img.height), (0, lp_img.height)]

matrix = cv2.getPerspectiveTransform(np.float32(box_points_lp), np.float32([point1, point2, point3, point4]))

transformed_lp_img = cv2.warpPerspective(np.array(lp_img), matrix, (640, 640))

transformed_lp_img = Image.fromarray(transformed_lp_img)

img.paste(transformed_lp_img, (0, 0), transformed_lp_img)
img.save('output.png')

[366.90540098 221.28711183] [638.14351638 247.57571799] [644.46771142 309.47836229] [373.22959602 283.18975613]
(371.81988355044456, 221.2871118319543) (643.0579989496235, 247.57571798721307) (649.3821939901359, 309.47836228761) (378.1440785909569, 283.18975613235125)


In [42]:
img = Image.open('output.png')
bounding_box = [(338.9159057409015, 132.80063567552827) ,(442.44545728802836, 142.6815241199347), (473.2059924349345, 167.71728808822513) ,(369.67644088780764, 157.83639964381868)]

size = 10
target_points = [(0, 0), (52, 0), (52, 11), (0, 11)]

matrix = cv2.getPerspectiveTransform(np.float32(bounding_box), np.float32(target_points))

transformed_lp_img = cv2.warpPerspective(np.array(img), matrix, (52, 11))

transformed_lp_img = Image.fromarray(transformed_lp_img)

transformed_lp_img = transformed_lp_img.resize((52 * size, 11 * size))

# # binarize the image
# transformed_lp_img = transformed_lp_img.convert('L')
# transformed_lp_img = transformed_lp_img.point(lambda x: 0 if x < 128 else 255, '1')



transformed_lp_img.show()



In [41]:
img = Image.open('test.jpg')
img = img.resize((640, 640))

box = [(360, 280), (600, 285), (600, 370), (350, 370)]

draw = ImageDraw.Draw(img)
draw.polygon(box, outline='red', width=3)

# img.show()

target_points = [(0, 0), (52, 0), (52, 11), (0, 11)]

matrix = cv2.getPerspectiveTransform(np.float32(box), np.float32(target_points))

transformed_lp_img = cv2.warpPerspective(np.array(img), matrix, (52, 11))

transformed_lp_img = Image.fromarray(transformed_lp_img)

transformed_lp_img = transformed_lp_img.resize((52 * size, 11 * size))

transformed_lp_img.show()
